In [2]:
import django, sys, os
sys.path.append('/home/galm/software/django/tmv/BasicBrowser/')
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

from sklearn.decomposition import NMF
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier

from scoping.models import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from itertools import product, combinations
from utils.text import *
from scipy.sparse import find

import math
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

from MulticoreTSNE import MulticoreTSNE as mTSNE
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import coverage_error, label_ranking_average_precision_score, label_ranking_loss
from pycountry_convert import  country_alpha2_to_continent_code, country_alpha3_to_country_alpha2
import pickle
import scipy.sparse
import datetime
import scipy.stats as st
import scoping.utils.learning_utils as lu
import matplotlib

django.db.connection.close()


In [3]:
pid = 178
qid = 8145
qid = 8609
q = Query.objects.get(pk=qid)
qs = Query.objects.filter(pk__in=[8145,7368,8272, 8696, 8609, 8696, 8777])

dids = set(Doc.objects.filter(query__in=qs).values_list('pk', flat=True))
p = Project.objects.get(pk=pid)

print(len(dids))

608037


In [4]:
#if pd.read_csv('../data/0c_doc_info.csv').shape[0] != len(dids):
df = pd.DataFrame.from_dict(Doc.objects.filter(pk__in=dids).values('id','PY','title','wosarticle__dt'))
df.head()
df.to_csv('../data/0c_doc_info.csv',index=False)

In [7]:
find_new_docs = False

if find_new_docs:
    old_q = Query.objects.get(pk=7368)

    new_docs = dids - set(Doc.objects.filter(query=q).values_list('pk',flat=True))

    old_docs = dids - new_docs

    print(len(old_docs))
    print(len(new_docs))

In [8]:
r_tag_ids = Tag.objects.get(query__project=p,title="random sample 1").doc_set.values_list('pk',flat=True)

len(r_tag_ids)

500

In [9]:
tag, created = Tag.objects.get_or_create(
    query=Query.objects.get(pk=8777),
    title="random sample 2"
)

if created:
    new_sample_size = len(r_tag_ids)/len(old_docs)*len(new_docs)

    new_sample_docs = random.sample(new_docs, round(new_sample_size))
    
    for did in new_sample_docs:
        Doc.objects.get(pk=did).tag.add(tag)

    tag.update_tag()

In [10]:


tag, created = Tag.objects.get_or_create(
    query=Query.objects.get(pk=8777),
    title="random sample 3"
)

if created:
    
    new_sample_docs = random.sample(dids, 250)

    for did in new_sample_docs:
        Doc.objects.get(pk=did).tag.add(tag)

    tag.update_tag()

In [13]:
for t in Tag.objects.filter(query__project=p).exclude(title='all').order_by('id').values('id','title'):
    print(t)

{'id': 4680, 'title': 'random sample 1'}
{'id': 4764, 'title': 'predicted relevant sample 2020-02-18 13:14'}
{'id': 5178, 'title': 'predicted relevant sample 2020-03-24 09:36'}
{'id': 5182, 'title': 'predicted relevant sample 2020-03-25 09:21'}
{'id': 5191, 'title': 'predicted Terrestrial ES sample 2020-03-25 15:17'}
{'id': 5192, 'title': 'predicted Mountains, snow and ice sample 2020-03-25 15:17'}
{'id': 5193, 'title': 'predicted Human and managed sample 2020-03-25 15:17'}
{'id': 5194, 'title': 'predicted Oceans and Coasts sample 2020-03-25 15:17'}
{'id': 5195, 'title': 'predicted Rivers lakes and moisture sample 2020-03-25 15:17'}
{'id': 5196, 'title': 'predicted climate system sample 2020-03-25 15:17'}
{'id': 5243, 'title': 'predicted relevant sample 2020-03-31 08:21'}
{'id': 5247, 'title': 'predicted relevant sample 2020-04-01 08:29'}
{'id': 5251, 'title': 'predicted 12 - Physical systems sample 2020-04-01 10:09:10.283298'}
{'id': 5252, 'title': 'predicted 12 - Rivers, lakes, and s

# Get documents from ar5 database, excluding those that look at alternative hypotheses than climate-impacts

In [9]:
# old_data = pd.read_csv('../data/category_predictions_cats.csv')
# old_data[old_data['ar5']==1].reset_index(drop=True).to_csv('../data/ar5_data.csv')
# print(old_data.shape)
# old_data.head()

In [10]:
# old_ar5 = set(old_data[old_data['ar5']==1].id.values)
# print(len(old_ar5))
# current = set(seen_df.id.values)
# missing_ar5 = old_ar5 - current

In [15]:
import statistics

django.db.connection.close()

def mode_na(x):
    try:
        return statistics.mode([y for y in x if y is not np.NaN])
    except:
        return np.NaN
    
columns = [
    'id',
    'content',
    'title',
    'wosarticle__de',
    'wosarticle__wc'
]

# Majority ratings
dos = DocOwnership.objects.filter(
    query__in=qs, 
    relevant__gt=0,
    doc__content__iregex='\w'
)
dodf = pd.DataFrame.from_dict(dos.order_by('finish').values(
    'id','doc_id','user__username','relevant'
))
dodf.loc[dodf['relevant']==2,'relevant'] = 0
dodf.loc[dodf['relevant']==3,'relevant'] = np.NaN

gdf = dodf.groupby('doc_id').agg(lambda x: mode_na(x)).reset_index()[['doc_id','relevant']].rename(columns={"doc_id":"id"})
gdf['seen'] = 1
gdf['ar5'] = 0
gdf.loc[pd.isna(gdf['relevant']),'seen'] = np.NaN

print(gdf[pd.notna(gdf['seen'])].shape)

# Ar5 docs
ar5_df = pd.read_csv('../data/ar5_data.csv')[['id']]
ar5_df['ar5'] = 1
ar5_df['seen'] = 1
ar5_df['relevant'] = 1
ar5_df.head()

seen_df = pd.concat([ar5_df,gdf])

ddf = pd.DataFrame.from_dict(
    Doc.objects.filter(pk__in=seen_df['id']).values(*columns)
)
seen_df = pd.merge(ddf,seen_df)

rep_tags = Tag.objects.filter(title__icontains="predicted relevant sample")
rep_tag_ids = set(DocOwnership.objects.filter(tag__in=rep_tags).values_list('doc__pk',flat=True))

r_tags = Tag.objects.filter(query__project=p, title__icontains="random sample")
#r_tag_ids = Tag.objects.get(query__project=p,title="random sample 1").doc_set.values_list('pk',flat=True)
r_tag_ids = set(DocOwnership.objects.filter(tag__in=r_tags).values_list('doc__pk',flat=True))
seen_df['random_sample'] = np.where(seen_df['id'].isin(r_tag_ids),1,0)
seen_df['representative_relevant_sample'] = np.where(seen_df['id'].isin(rep_tag_ids),1,0)

physical_tags = Tag.objects.filter(query__in=qs,id__gt=5177)
pdos = DocOwnership.objects.filter(tag__in=physical_tags,relevant__gt=0).values_list('doc__id',flat=True)
seen_df['physical_tags'] = np.where(seen_df['id'].isin(pdos),1,0)

seen_df = seen_df[pd.notna(seen_df['seen'])]

print(seen_df.shape)
seen_df.head()


(2374, 4)
(2630, 11)


,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,representative_relevant_sample,physical_tags
0,770076,Climate is well known as an important determin...,Present-day and future contribution of climate...,boreal forests; China; climate change; dynamic...,[Ecology],0,1.0,1.0,0,0,1
1,1283572,We review recent climate changes over the Tibe...,"Climatic and associated cryospheric, biospheri...",climate change; Tibetan Plateau; cryosphere; b...,[Meteorology & Atmospheric Sciences],0,1.0,1.0,0,1,1
2,1692237,Devastating floods adversely affect human life...,Impact of Climate Change on Flood Frequency an...,Kabul basin; climate-change; GCMs; SWAT model;...,"[Geosciences, Multidisciplinary]",0,1.0,0.0,0,0,1
3,440304,"The common cutworm, Spodoptera litura, has bec...",Temperature Impacts the Development and Surviv...,None,[Multidisciplinary Sciences],0,1.0,0.0,0,0,1
4,1286327,This study investigates trends and the effects...,"Extreme rainfall indices in Distrito Federal, ...",Brazil; Distrito Federal; ENSO; extreme indice...,[Meteorology & Atmospheric Sciences],0,1.0,1.0,0,0,1


## Get AR5 categories

In [16]:
cats = Category.objects.filter(project=p,level=12)
early_ids = set(DocOwnership.objects.filter(
    query__in=qs,tag__lt=4765
).values_list('doc__pk',flat=True))
ar5_ids = ar5_df.id
ldf = pd.DataFrame.from_dict(list(Doc.objects.filter(pk__in=set(ar5_ids)|early_ids).filter(
    category__in=cats
).values('id','category__level','category__name')))
ldf['cname'] =  ldf['category__level'].astype(str) + " - " + ldf['category__name']
ldf = ldf.drop(columns=['category__level','category__name'])
ldf['v'] = 1

cat_map = ldf.pivot(index='id',columns="cname", values="v")
cat_map['id'] = cat_map.index
cat_map = cat_map.reset_index(drop=True)

seen_df = pd.merge(seen_df, cat_map, how="left").fillna(0)

## Get majority category decision

In [17]:
dudf = pd.DataFrame.from_dict(
    DocUserCat.objects.filter(
        doc__pk__in=seen_df['id'],category__project=p
    ).values('user__username','doc__id','category__level','category__name')
)
dudf['v'] = 1
dudf['cname'] =  dudf['category__level'].astype(str) + " - " + dudf['category__name']#
dudf = dudf.drop(columns=['category__level','category__name'])
dudf_wide = dudf.pivot_table(index=['doc__id','user__username'],columns="cname", values="v").fillna(0)
print(dudf_wide.shape)
gudf = dudf_wide.groupby(['doc__id']).agg(lambda x: st.mode(x)[0][0]).reset_index()
dudf_wide_r = dudf_wide.reset_index()
dudf_wide_r.columns = ["doc__id",'user__username'] + list(dudf_wide.columns)
dudf_expanded = dudf_wide_r.melt(id_vars=["doc__id","user__username"]).head()
print(gudf.shape)
gudf.tail()

(1517, 233)
(1186, 234)


cname,doc__id,10 - places,11 - Africa,11 - Asia,11 - Australasia,11 - Central & South America,11 - Europe,11 - Global,11 - North America,11 - Open ocean,...,8 - 8.03. Quantitative - process-based modelling,8 - 8.04. Quantitative - empirical models,8 - 8.05. Quantitative - statistical,8 - 8.06. Qualitative - literature review,8 - 8.07. Qualitative - expert interviews,8 - 8.08. Qualitative - household surveys,8 - 8.09. Mixed methods,8 - 8.10. Case studies,8 - 8.11. Remote sensing,9 - observations
1181,3897103,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1182,3917670,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1183,3977439,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1184,3978679,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1185,3981306,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
from alterations import postfix_data

seen_df = pd.merge(seen_df, gudf.rename(columns={"doc__id":"id"}), how="left").fillna(0)
print(seen_df.shape)

seen_df = postfix_data(seen_df)
seen_df.to_csv('../data/0_labelled_documents.csv', index=False)
seen_df.head()

(2630, 244)


,id,content,title,wosarticle__de,wosarticle__wc,ar5,seen,relevant,random_sample,representative_relevant_sample,...,8 - 8.05. Quantitative - statistical,8 - 8.06. Qualitative - literature review,8 - 8.07. Qualitative - expert interviews,8 - 8.08. Qualitative - household surveys,8 - 8.09. Mixed methods,8 - 8.10. Case studies,8 - 8.11. Remote sensing,9 - observations,6 - 72 Conflict,4 - 50 Other (marine & coastal)
0,770076,Climate is well known as an important determin...,Present-day and future contribution of climate...,boreal forests; China; climate change; dynamic...,[Ecology],0,1.0,1.0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,1283572,We review recent climate changes over the Tibe...,"Climatic and associated cryospheric, biospheri...",climate change; Tibetan Plateau; cryosphere; b...,[Meteorology & Atmospheric Sciences],0,1.0,1.0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
2,1692237,Devastating floods adversely affect human life...,Impact of Climate Change on Flood Frequency an...,Kabul basin; climate-change; GCMs; SWAT model;...,"[Geosciences, Multidisciplinary]",0,1.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,440304,"The common cutworm, Spodoptera litura, has bec...",Temperature Impacts the Development and Surviv...,0,[Multidisciplinary Sciences],0,1.0,0.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,1286327,This study investigates trends and the effects...,"Extreme rainfall indices in Distrito Federal, ...",Brazil; Distrito Federal; ENSO; extreme indice...,[Meteorology & Atmospheric Sciences],0,1.0,1.0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:
unseen_docs = Doc.objects.filter(
    id__in=dids,
    content__iregex='\w',
).exclude(pk__in=seen_df['id']).distinct('id')

unseen_df = pd.DataFrame.from_dict(list(unseen_docs.values(*columns)))
unseen_df = unseen_df[columns]
unseen_df['relevant'] = 0
unseen_df['seen'] = 0
unseen_df['random_sample'] = 0

print(unseen_df.shape)
unseen_df.to_csv('../data/0_unlabelled_documents.csv', index=False)
unseen_df.head()

In [ ]:
django.db.connection.close()
df = (pd.concat([seen_df,unseen_df])
      .sort_values('id')
      .sample(frac=1, random_state=1)
      .reset_index(drop=True)
)
y = df['relevant']

seen_index = df[df['seen']==1].index
unseen_index = df[df['seen']==0].index
new_index = df[(df['seen']==1) & (df['ar5']==0)].index
rel_index = df[df['relevant']==1].index

r_index = df[df["random_sample"]==1].index
print(df.shape)
df.head()